In [1]:
import pandas as pd
from collections import defaultdict
import warnings
import visualize
import neat
import multiprocessing
import os
import matplotlib.pyplot as plt

import numpy as np

In [2]:
necoc = 30
generations = int(3000/necoc)
generations

100

In [3]:
def softmax(x, axis=1):
    # 计算每行的最大值
    row_max = x.max(axis=axis)
 
    # 每行元素都需要减去对应的最大值，否则求exp(x)会溢出，导致inf情况
    row_max=row_max.reshape(-1, 1)
    x = x - row_max
 
    # 计算e的指数次幂
    x_exp = np.exp(x)
    x_sum = np.sum(x_exp, axis=axis, keepdims=True)
    s = x_exp / x_sum
    return s

In [4]:
df = pd.read_csv('letter-recognition.data',header=None)

train_df = df[:1600]
test_df = df[16000:16000+400]

X = train_df.ix[:,1:16].to_numpy() 
y = np.array([ord(l)-ord('A') for l in train_df.ix[:,0].to_numpy()])

In [5]:
def get_winner(class1):
    
    numbers = [*range(26)] # number 2 > number 1
    number_of_classification = len(numbers)
    #number_of_sampling = 100

    list_y = y.tolist()
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

        
    class2 = []

    class1_indexes = []
    class2_indexes = []

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)

    print("number of class1:{}".format(len(class1_label)))
    print("number of class2:{}".format(len(class2_label)))

    samplesize = 1600
    x_inputs =[tuple(c) for c in X[class1_indexes].tolist()] + [tuple(c) for c in X[class2_indexes].tolist()]
    x_outputs = [tuple([c]) for c in class1_label + class2_label]
    
    
    def eval_genomes(genomes, config):
        for genome_id, genome in genomes:
            net = neat.nn.FeedForwardNetwork.create(genome, config)

            outputs = []
            for xi in x_inputs:
                output = net.activate(xi)
                outputs.append(output)


            px_outputs = softmax(np.array(outputs).reshape(samplesize, 2), axis=1)
            # the index of maximum in each line
            pred_outputs = np.argmax(px_outputs, axis = 1)
            real_outputs = np.array(x_outputs).reshape(samplesize,)

            acc = np.sum(pred_outputs == real_outputs)/samplesize

            genome.fitness = acc

    def run(config_file):
        # Load configuration.
        config = neat.Config(
            neat.DefaultGenome,
            neat.DefaultReproduction,
            neat.DefaultSpeciesSet,
            neat.DefaultStagnation,
            config_file,
        )

        # Create the population, which is the top-level object for a NEAT run.
        p = neat.Population(config)

        # add a stdout reporter to show progress in the terminal
        reporter = neat.StdOutReporter(False)
        p.add_reporter(reporter)
        stats = neat.StatisticsReporter()
        p.add_reporter(stats)
        #checkpointer = neat.Checkpointer(100)
        #p.add_reporter(checkpointer)
        # Run for up to 300 generations.
        winner = p.run(eval_genomes, generations)

        return [stats, winner]
    
    local_dir = os.getcwd()
    config_path = os.path.join(local_dir, "config-feedforward-letter2")
    [stats, winner] = run(config_path)
    
    return winner

In [6]:
import numpy as np
import random

number_codes = []
for i in range(2**(26-1), 2**26-1):
    if (i % 1000000) == 0:
            print(i)
    for j in bin(i).split('b')[1]:
        number_codes.append(int(j))
        


number_codes = np.array(number_codes).reshape(2**(26 - 1) - 1,26).T     

slice = random.sample(range(2**(26 - 1) - 1), necoc)

number_codes = number_codes[:,slice].tolist()

classes = []
for i in range(necoc):
    code_list = np.array(number_codes).T[i]
    classes.append(np.where(code_list==1)[0].tolist())

matrix_errors = []

for i in range(25):
    for j in range(i+1,26):
        matrix_errors.append(necoc - np.sum(np.array(number_codes[i]) == np.array(number_codes[j])))
min_error = np.min(matrix_errors)

print(min_error)

34000000
35000000
36000000
37000000
38000000
39000000
40000000
41000000
42000000
43000000
44000000
45000000
46000000
47000000
48000000
49000000
50000000
51000000
52000000
53000000
54000000
55000000
56000000
57000000
58000000
59000000
60000000
61000000
62000000
63000000
64000000
65000000
66000000
67000000
7


In [7]:
winner_list = []

for i,class1 in enumerate(classes):
    print("running:{}/30".format(i))
    winner = get_winner(class1)
    winner_list.append(winner)

running:0/30
number of class1:1058
number of class2:542

 ****** Running generation 0 ****** 

Population's average fitness: 0.49443 stdev: 0.14734
Best fitness: 0.73562 - size: (2, 3) - species 4 - id 156
Average adjusted fitness: 0.220
Mean genetic distance 2.698, standard deviation 0.499
Population of 200 members in 14 species
Total extinctions: 0
Generation time: 3.236 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.55413 stdev: 0.13785
Best fitness: 0.73562 - size: (2, 3) - species 5 - id 156
Average adjusted fitness: 0.233
Mean genetic distance 2.779, standard deviation 0.519
Population of 203 members in 16 species
Total extinctions: 0
Generation time: 3.841 sec (3.538 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.57482 stdev: 0.13252
Best fitness: 0.73562 - size: (2, 3) - species 1 - id 156
Average adjusted fitness: 0.282
Mean genetic distance 2.840, standard deviation 0.552
Population of 202 members in 18 species
Tota

Population's average fitness: 0.57941 stdev: 0.13675
Best fitness: 0.73938 - size: (8, 14) - species 7 - id 3537
Average adjusted fitness: 0.285
Mean genetic distance 3.050, standard deviation 0.695
Population of 197 members in 11 species
Total extinctions: 0
Generation time: 5.682 sec (5.617 average)

 ****** Running generation 25 ****** 

Population's average fitness: 0.56980 stdev: 0.14375
Best fitness: 0.73938 - size: (8, 14) - species 7 - id 3537
Average adjusted fitness: 0.282
Mean genetic distance 2.955, standard deviation 0.697
Population of 201 members in 9 species
Total extinctions: 0
Generation time: 5.965 sec (5.644 average)

 ****** Running generation 26 ****** 

Population's average fitness: 0.55557 stdev: 0.14959
Best fitness: 0.73938 - size: (8, 14) - species 7 - id 3537
Average adjusted fitness: 0.245
Mean genetic distance 2.972, standard deviation 0.652
Population of 200 members in 9 species
Total extinctions: 0
Generation time: 5.627 sec (5.678 average)

 ****** Runn

Population's average fitness: 0.57300 stdev: 0.14376
Best fitness: 0.75813 - size: (10, 18) - species 7 - id 8121
Average adjusted fitness: 0.237
Mean genetic distance 2.941, standard deviation 0.618
Population of 201 members in 9 species
Total extinctions: 0
Generation time: 6.165 sec (6.554 average)

 ****** Running generation 49 ****** 

Population's average fitness: 0.58359 stdev: 0.13964
Best fitness: 0.75813 - size: (10, 18) - species 7 - id 8121
Average adjusted fitness: 0.251
Mean genetic distance 2.755, standard deviation 0.587
Population of 201 members in 9 species
Total extinctions: 0
Generation time: 5.702 sec (6.441 average)

 ****** Running generation 50 ****** 

Population's average fitness: 0.58014 stdev: 0.14442
Best fitness: 0.75813 - size: (10, 18) - species 7 - id 8121
Average adjusted fitness: 0.281
Mean genetic distance 2.706, standard deviation 0.599
Population of 201 members in 9 species
Total extinctions: 0
Generation time: 5.598 sec (6.329 average)

 ****** Ru

Population's average fitness: 0.60003 stdev: 0.14006
Best fitness: 0.75813 - size: (10, 18) - species 7 - id 8121
Average adjusted fitness: 0.305
Mean genetic distance 2.921, standard deviation 0.585
Population of 197 members in 16 species
Total extinctions: 0
Generation time: 5.189 sec (5.390 average)

 ****** Running generation 73 ****** 

Population's average fitness: 0.60836 stdev: 0.13759
Best fitness: 0.75813 - size: (10, 18) - species 7 - id 8121
Average adjusted fitness: 0.314
Mean genetic distance 2.935, standard deviation 0.595
Population of 198 members in 16 species
Total extinctions: 0
Generation time: 5.197 sec (5.384 average)

 ****** Running generation 74 ****** 

Population's average fitness: 0.59771 stdev: 0.14537
Best fitness: 0.75813 - size: (10, 18) - species 7 - id 8121
Average adjusted fitness: 0.299
Mean genetic distance 2.983, standard deviation 0.588
Population of 201 members in 14 species
Total extinctions: 0
Generation time: 5.371 sec (5.379 average)

 ******

Population's average fitness: 0.59436 stdev: 0.15252
Best fitness: 0.76313 - size: (11, 23) - species 30 - id 13116
Average adjusted fitness: 0.262
Mean genetic distance 3.107, standard deviation 0.603
Population of 199 members in 18 species
Total extinctions: 0
Generation time: 6.799 sec (7.130 average)

 ****** Running generation 97 ****** 

Population's average fitness: 0.60347 stdev: 0.14550
Best fitness: 0.76313 - size: (11, 23) - species 30 - id 13116
Average adjusted fitness: 0.271
Mean genetic distance 3.117, standard deviation 0.597
Population of 199 members in 18 species
Total extinctions: 0
Generation time: 6.981 sec (7.151 average)

 ****** Running generation 98 ****** 

Population's average fitness: 0.61311 stdev: 0.13641
Best fitness: 0.76313 - size: (11, 23) - species 30 - id 13116
Average adjusted fitness: 0.285
Mean genetic distance 3.044, standard deviation 0.573
Population of 201 members in 18 species
Total extinctions: 0
Generation time: 6.954 sec (7.080 average)

 

...

Population's average fitness: 0.52984 stdev: 0.03837
Best fitness: 0.62062 - size: (15, 31) - species 15 - id 11368
Average adjusted fitness: 0.101
Mean genetic distance 3.206, standard deviation 0.584
Population of 200 members in 15 species
Total extinctions: 0
Generation time: 8.074 sec (7.522 average)

 ****** Running generation 71 ****** 

Population's average fitness: 0.52821 stdev: 0.03899
Best fitness: 0.62062 - size: (15, 31) - species 41 - id 11368
Average adjusted fitness: 0.101
Mean genetic distance 3.242, standard deviation 0.550
Population of 201 members in 18 species
Total extinctions: 0
Generation time: 8.080 sec (7.567 average)

 ****** Running generation 72 ****** 

Population's average fitness: 0.53285 stdev: 0.03895
Best fitness: 0.62062 - size: (15, 31) - species 41 - id 11368
Average adjusted fitness: 0.065
Mean genetic distance 3.260, standard deviation 0.529
Population of 199 members in 23 species
Total extinctions: 0
Generation time: 8.125 sec (7.633 average)

 

Population's average fitness: 0.53944 stdev: 0.04158
Best fitness: 0.62062 - size: (15, 31) - species 41 - id 11368
Average adjusted fitness: 0.120
Mean genetic distance 3.320, standard deviation 0.566
Population of 196 members in 18 species
Total extinctions: 0
Generation time: 9.802 sec (9.285 average)

 ****** Running generation 95 ****** 

Population's average fitness: 0.53730 stdev: 0.03956
Best fitness: 0.62062 - size: (15, 31) - species 41 - id 11368
Average adjusted fitness: 0.061
Mean genetic distance 3.290, standard deviation 0.568
Population of 194 members in 17 species
Total extinctions: 0
Generation time: 9.069 sec (9.296 average)

 ****** Running generation 96 ****** 

Population's average fitness: 0.53773 stdev: 0.03993
Best fitness: 0.62062 - size: (15, 31) - species 41 - id 11368
Average adjusted fitness: 0.077
Mean genetic distance 3.298, standard deviation 0.574
Population of 201 members in 17 species
Total extinctions: 0
Generation time: 9.277 sec (9.331 average)

 

Population's average fitness: 0.58419 stdev: 0.11016
Best fitness: 0.68937 - size: (6, 9) - species 30 - id 1360
Average adjusted fitness: 0.239
Mean genetic distance 3.168, standard deviation 0.541
Population of 193 members in 35 species
Total extinctions: 0
Generation time: 5.067 sec (4.879 average)

 ****** Running generation 19 ****** 

Population's average fitness: 0.59866 stdev: 0.09960
Best fitness: 0.68937 - size: (6, 9) - species 30 - id 1360
Average adjusted fitness: 0.251
Mean genetic distance 3.164, standard deviation 0.536
Population of 193 members in 34 species
Total extinctions: 0
Generation time: 4.711 sec (4.899 average)

 ****** Running generation 20 ****** 

Population's average fitness: 0.59230 stdev: 0.10223
Best fitness: 0.68937 - size: (6, 9) - species 30 - id 1360
Average adjusted fitness: 0.242
Mean genetic distance 3.200, standard deviation 0.544
Population of 191 members in 30 species
Total extinctions: 0
Generation time: 4.972 sec (4.921 average)

 ****** Ru

Population's average fitness: 0.58466 stdev: 0.10758
Best fitness: 0.70125 - size: (5, 12) - species 13 - id 5897
Average adjusted fitness: 0.229
Mean genetic distance 3.284, standard deviation 0.566
Population of 197 members in 18 species
Total extinctions: 0
Generation time: 6.737 sec (6.230 average)

 ****** Running generation 43 ****** 

Population's average fitness: 0.58443 stdev: 0.11211
Best fitness: 0.70125 - size: (5, 12) - species 13 - id 5897
Average adjusted fitness: 0.254
Mean genetic distance 3.259, standard deviation 0.573
Population of 193 members in 19 species
Total extinctions: 0
Generation time: 6.634 sec (6.326 average)

 ****** Running generation 44 ****** 

Population's average fitness: 0.58115 stdev: 0.11541
Best fitness: 0.70125 - size: (5, 12) - species 13 - id 5897
Average adjusted fitness: 0.260
Mean genetic distance 3.265, standard deviation 0.582
Population of 193 members in 19 species
Total extinctions: 0
Generation time: 6.350 sec (6.387 average)

 ******

Generation time: 6.479 sec (6.687 average)

 ****** Running generation 67 ****** 

Population's average fitness: 0.57816 stdev: 0.11151
Best fitness: 0.71125 - size: (7, 14) - species 13 - id 10171
Average adjusted fitness: 0.218
Mean genetic distance 3.063, standard deviation 0.610
Population of 203 members in 14 species
Total extinctions: 0
Generation time: 6.706 sec (6.643 average)

 ****** Running generation 68 ****** 

Population's average fitness: 0.57405 stdev: 0.11340
Best fitness: 0.71125 - size: (7, 14) - species 13 - id 10171
Average adjusted fitness: 0.210
Mean genetic distance 3.134, standard deviation 0.608
Population of 197 members in 14 species
Total extinctions: 0
Generation time: 6.645 sec (6.640 average)

 ****** Running generation 69 ****** 

Population's average fitness: 0.56278 stdev: 0.11987
Best fitness: 0.71125 - size: (7, 14) - species 13 - id 10171
Average adjusted fitness: 0.206
Mean genetic distance 3.118, standard deviation 0.594
Population of 199 members 

In [8]:
testX = test_df.ix[:,1:16].to_numpy() 
testy = np.array([ord(l)-ord('A') for l in test_df.ix[:,0].to_numpy()])

local_dir = os.getcwd()
config_path = os.path.join(local_dir, "config-feedforward-letter2")

config = neat.Config(
    neat.DefaultGenome,
    neat.DefaultReproduction,
    neat.DefaultSpeciesSet,
    neat.DefaultStagnation,
    config_path,
)

def get_pred_real(i, j):
    winner_net = neat.nn.FeedForwardNetwork.create(winner_list[i], config)
    
    numbers = [*range(26)]
    class1 = classes[i]
    class2 = []
    class1_indexes = []
    class2_indexes = []
    
    test_X = testX[j]
    test_y = testy[j]

    list_y = [test_y]
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)


    testsamplesize = 1
    test_x_inputs =[tuple(test_X)]
    test_x_outputs = [tuple([c]) for c in class1_label + class2_label]

    outputs = []
    for xi in test_x_inputs:
        output = winner_net.activate(xi)
        outputs.append(output)

    px_outputs = softmax(np.array(outputs).reshape(testsamplesize, 2), axis=1)
    # the index of maximum in each line
    pred_outputs = np.argmax(px_outputs, axis = 1)
    real_outputs = np.array(test_x_outputs).reshape(testsamplesize,)

    
    return [pred_outputs, real_outputs]

from random import randint
pred_value = []
error_list = []

for j in range(400):
    pred = []
    for i in range(necoc):
        [pred_outputs, real_outputs] = get_pred_real(i, j)
        #print(pred_outputs, real_outputs)
        pred.append(pred_outputs)
    #print(np.array(pred).T)
    
    error = []
    for i in range(26):
        error.append(necoc - np.sum(number_codes[i] == np.array(pred).T))
    #print(error)
    
    pred_value.append(np.where(error==np.min(error)))
    error_list.append(np.min(error))
    #print(np.where(error==np.min(error)) )
list_P = []
for i in pred_value:
    if (len(i[0])) == 1:
        list_P.append(i[0][0])
    else:
        random_pick = randint(0, len(i[0])-1)
        list_P.append(i[0][random_pick])
np.sum(list_P == testy)/400

0.2925

In [9]:
def get_pred_train(i, j):
    winner_net = neat.nn.FeedForwardNetwork.create(winner_list[i], config)
    
    numbers = [*range(26)]
    class1 = classes[i]
    class2 = []
    class1_indexes = []
    class2_indexes = []
    
    test_X = X[j]
    test_y = y[j]

    list_y = [test_y]
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)


    testsamplesize = 1
    test_x_inputs =[tuple(test_X)]
    test_x_outputs = [tuple([c]) for c in class1_label + class2_label]

    outputs = []
    for xi in test_x_inputs:
        output = winner_net.activate(xi)
        outputs.append(output)

    px_outputs = softmax(np.array(outputs).reshape(testsamplesize, 2), axis=1)
    # the index of maximum in each line
    pred_outputs = np.argmax(px_outputs, axis = 1)
    real_outputs = np.array(test_x_outputs).reshape(testsamplesize,)

    
    return [pred_outputs, real_outputs]

pred_value = []
error_list = []

for j in range(1600):
    pred = []
    for i in range(necoc):
        [pred_outputs, real_outputs] = get_pred_train(i, j)
        #print(pred_outputs, real_outputs)
        pred.append(pred_outputs)
    #print(np.array(pred).T)
    
    error = []
    for i in range(26):
        error.append(26 - np.sum(number_codes[i] == np.array(pred).T))
    #print(error)
    
    pred_value.append(np.where(error==np.min(error)))
    error_list.append(np.min(error))
    #print(np.where(error==np.min(error)) )

list_P = []
for i in pred_value:
    if (len(i[0])) == 1:
        list_P.append(i[0][0])
    else:
        random_pick = randint(0, len(i[0])-1)
        list_P.append(i[0][random_pick])
print("Trainset acc:{}".format(np.sum(list_P == y)/1600))
winner_fitness = []
for winner in winner_list:
    #print(winner.fitness)
    winner_fitness.append(winner.fitness)
print("Avg Base Learner:{}".format(np.mean(winner_fitness)))

Trainset acc:0.36125
Avg Base Learner:0.7024166666666667


In [10]:
# program to check if there is exist a path between two vertices 
# of a graph 

from collections import defaultdict 

#This class represents a directed graph using adjacency list representation 
class Graph: 

	def __init__(self,vertices): 
		self.V= vertices #No. of vertices 
		self.graph = defaultdict(list) # default dictionary to store graph 

	# function to add an edge to graph 
	def addEdge(self,u,v): 
		self.graph[u].append(v) 
	
	# Use BFS to check path between s and d 
	def isReachable(self, s, d): 
		# Mark all the vertices as not visited 
		visited =[False]*(self.V) 

		# Create a queue for BFS 
		queue=[] 

		# Mark the source node as visited and enqueue it 
		queue.append(s) 
		visited[s] = True

		while queue: 

			#Dequeue a vertex from queue 
			n = queue.pop(0) 
			
			# If this adjacent node is the destination node, 
			# then return true 
			if n == d: 
				return True

			# Else, continue to do BFS 
			for i in self.graph[n]: 
				if visited[i] == False: 
					queue.append(i) 
					visited[i] = True
		# If BFS is complete without visited d 
		return False
    
    
def findAllPath(graph,start,end,path=[]):
    path = path +[start]
    if start == end:
        return [path]
 
    paths = [] #存储所有路径    
    for node in graph[start]:
        if node not in path:
            newpaths = findAllPath(graph,node,end,path) 
            for newpath in newpaths:
                paths.append(newpath)
    return paths

numbers = [*range(2)]
number_of_classification = len(numbers)

list_nodes_number = []
list_connection_number = []

for winner in winner_list:
    used_nodes = list(winner.nodes.keys())
    # create a dict for mapping 
    l1 = numbers + config.genome_config.input_keys + used_nodes[number_of_classification:]
    l2 = range(len(l1))
    dict_nodes = dict(zip(l1, l2))

    ### 将节点加入图
    g = Graph(len(l1))# inputs + outputs + used  
    # add connections in the graph
    for cg in winner.connections.values():
        if cg.enabled:
            g.addEdge(dict_nodes[cg.key[0]],dict_nodes[cg.key[1]])

    v = numbers
    list_connections = []
    for vi in v:
        for u in range(number_of_classification, len(config.genome_config.input_keys) + number_of_classification):
            list_connections.append(g.isReachable(u, vi))

    nodes_inputs = range(number_of_classification, number_of_classification+ 64)
    nodes_outputs = range(number_of_classification)
    nodes_mid = range(number_of_classification + 64, len(dict_nodes))

    # 找到所有输入和输出连接的路
    all_path = []
    for u in nodes_inputs:
        for v in nodes_outputs:
            path = findAllPath(g.graph, u, v)
            if path:
                all_path = all_path + path

    # 得到最长路，以及每个节点在哪个层级
    max_length = max([len(x) for x in all_path])
    nodes_tuples_list = []
    for path in all_path:
        #print(path)
        for node in path:
            nodes_tuples_list.append([node, path.index(node)])

    # 确定节点的层级（消除重复）       
    nodes_tuples_fixed_list = []
    for index in range(max_length):
        for nodes in nodes_tuples_list:
            node, node_index = nodes[0], nodes[1]
            if node_index == index: 
                if nodes not in nodes_tuples_fixed_list:
                    nodes_tuples_fixed_list.append(nodes)


    # 所有有连接的节点
    all_used_nodes = []
    for node in nodes_tuples_fixed_list:
        if node[0] not in all_used_nodes:
            all_used_nodes.append(node[0])

    # 确定节点的层级（最终确定）
    nodes_tuples_fixed_list_final = []

    for node in all_used_nodes:
        if node in nodes_inputs:
            nodes_tuples_fixed_list_final.append([node, 0])
        elif node in nodes_outputs:
            nodes_tuples_fixed_list_final.append([node, max_length - 1])
        else:
            list_node = []
            for nodes in nodes_tuples_fixed_list:
                if node == nodes[0]:
                    list_node.append(nodes)
            layer = 0
            for ele in list_node:
                # 删除重复项，取最大
                if ele[1] > layer:
                    layer = ele[1]
            nodes_tuples_fixed_list_final.append([node,layer])


    # 每个层级的节点合数
    length_of_layers = []
    number_of_nodes = 0
    for index in range(max_length):
        for nodes in nodes_tuples_fixed_list_final:
            node, node_index = nodes[0], nodes[1]
            if node_index == index: 
                number_of_nodes += 1
        length_of_layers.append(number_of_nodes)

    length_of_each_layer = []
    length_of_each_layer.append(length_of_layers[0])
    for i in range(1, max_length):
        length_of_each_layer.append(length_of_layers[i] - length_of_layers[i-1])

    # 输出每个层级的节点个数
    #print("length of each layers:", length_of_each_layer)

    # 所有端到端的路
    all_path_side2side = []
    for path in all_path:
        if len(path) == 2:
            all_path_side2side.append(path)
        else:
            for i in range(len(path)-1):
                all_path_side2side.append([path[i],path[i+1]])

    # 定义节点到绘图的字典
    dict_nodes_graph = dict()
    count_number_layer = [0] * max_length
    for nodes in nodes_tuples_fixed_list_final:
        node, layer = nodes
        dict_nodes_graph[node] = [count_number_layer[layer], layer]
        count_number_layer[layer] += 1
    list_nodes_number.append(np.sum(length_of_each_layer))
    list_connection_number.append(len(all_path_side2side))
    #print("Number of nodes:{} Number of connections:{}".format(np.sum(length_of_each_layer),len(all_path_side2side)))
print("Total nodes:{} Total connections::{}".format(np.sum(list_nodes_number), np.sum(list_connection_number)))

Total nodes:490 Total connections::3934
